# Face Verification - Model Inference Testing

This notebook demonstrates face verification using pretrained FaceNet models. We'll cover:

1. **Model Setup** - Loading pretrained FaceNet and MTCNN models
2. **Single Image Testing** - Face detection and embedding generation
3. **Face Verification** - Comparing two faces for similarity
4. **Batch Processing** - Verifying multiple image pairs
5. **Performance Analysis** - Evaluating verification accuracy

## Prerequisites

Before running this notebook, ensure you have:
- Installed all required packages from requirements.txt
- Preprocessed images in the data/processed directory
- Stable internet connection (for downloading pretrained models on first run)

In [ ]:
# Import required libraries
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from pathlib import Path
import time

# Add project source directory to path
project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))

# Import custom modules
from inference import FaceVerifier, load_face_verifier, quick_verify
from preprocessing import load_image, preprocess_image
from utils import setup_project_environment, VisualizationUtils

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

# Setup project environment
paths, logger, config = setup_project_environment()

# Check device availability
if torch.cuda.is_available():
    device_info = f"CUDA ({torch.cuda.get_device_name()})"
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device_info = "Apple Silicon GPU (MPS)"
else:
    device_info = "CPU"

print(f"🖥️  Available device: {device_info}")
print(f"🎯 PyTorch version: {torch.__version__}")
print("✅ Setup complete!")

## 1. Khởi tạo Face Verifier

Khởi tạo mô hình FaceNet và MTCNN để thực hiện face verification.

In [ ]:
# Khởi tạo Face Verifier
print("🚀 Đang khởi tạo FaceVerifier...")
print("=" * 50)

# Khởi tạo verifier (sẽ tải model nếu chưa có)
verifier = FaceVerifier()

print("\n✅ FaceVerifier đã khởi tạo thành công!")
print(f"📱 Device: {verifier.device}")
print(f"🎯 Detection threshold: {verifier.detection_threshold}")
print(f"🔍 Verification threshold: {verifier.verification_threshold}")

# Kiểm tra khả năng của thiết bị
if verifier.device == 'mps':
    print("🍎 Đang sử dụng Apple Silicon GPU - hiệu suất tối ưu!")
elif verifier.device == 'cuda':
    print("🚀 Đang sử dụng NVIDIA GPU - hiệu suất cao!")
else:
    print("💻 Đang sử dụng CPU - vẫn hoạt động tốt!")

print("\n🎉 Mô hình sẵn sàng cho face verification!")

## 2. Tạo ảnh mẫu để test

Vì thư mục `data/raw/` hiện tại chưa có ảnh thật, chúng ta sẽ tạo một số ảnh mẫu để test hệ thống.

In [ ]:
# Tạo ảnh mẫu với khuôn mặt giả để test
def create_sample_faces():
    """Tạo ảnh mẫu có hình dạng giống khuôn mặt để test face detection"""
    
    sample_images = []
    
    for i in range(3):
        # Tạo ảnh với kích thước 224x224
        img = np.random.randint(50, 200, (224, 224, 3), dtype=np.uint8)
        
        # Thêm vùng sáng hơn giống khuôn mặt
        center_x, center_y = 112, 112
        face_size = 80
        
        # Vẽ oval giống khuôn mặt
        for y in range(224):
            for x in range(224):
                # Tính khoảng cách từ center
                dist_x = (x - center_x) / face_size
                dist_y = (y - center_y) / (face_size * 1.3)  # Oval hơn
                
                if dist_x**2 + dist_y**2 <= 1:
                    # Khu vực khuôn mặt - màu sáng hơn
                    img[y, x] = img[y, x] * 0.3 + np.array([220, 180, 150]) * 0.7
                    
                    # Thêm mắt
                    if (abs(x - (center_x - 20)) < 8 and abs(y - (center_y - 15)) < 5) or \
                       (abs(x - (center_x + 20)) < 8 and abs(y - (center_y - 15)) < 5):
                        img[y, x] = [50, 50, 50]  # Màu tối cho mắt
                    
                    # Thêm miệng
                    if abs(x - center_x) < 15 and abs(y - (center_y + 25)) < 3:
                        img[y, x] = [100, 50, 50]  # Màu đỏ nhạt cho miệng
        
        sample_images.append(img)
    
    return sample_images

# Tạo ảnh mẫu
print("🎨 Đang tạo ảnh mẫu để test...")
sample_faces = create_sample_faces()

# Lưu ảnh mẫu
sample_paths = []
for i, img in enumerate(sample_faces):
    filename = f"sample_face_{i+1}.jpg"
    filepath = paths.data_raw / filename
    
    # Chuyển từ RGB sang BGR cho OpenCV
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(str(filepath), img_bgr)
    sample_paths.append(str(filepath))
    
    print(f"✅ Đã tạo: {filename}")

print(f"\n📁 Đã tạo {len(sample_faces)} ảnh mẫu trong: {paths.data_raw}")

# Hiển thị ảnh mẫu
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for i, img in enumerate(sample_faces):
    axes[i].imshow(img)
    axes[i].set_title(f'Ảnh mẫu {i+1}')
    axes[i].axis('off')

plt.suptitle('Ảnh mẫu để test Face Detection', fontsize=14)
plt.tight_layout()
plt.show()

print("💡 Lưu ý: Đây chỉ là ảnh mẫu để test. Thay thế bằng ảnh khuôn mặt thật để có kết quả tốt hơn!")